<a href="https://colab.research.google.com/github/jvataidee/ManualdeAnalisedeDados_Python/blob/master/cap3_estatistica_descritiva_bivariada.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**by: [João Ataíde](https://www.joaoataide.com)**
# **CAP 02: Estatística Descritiva Bivariada**

---



In [1]:
#importar bibliotecas
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
import seaborn as sns
import statistics as st
import scipy.stats as stats

## Associações entre duas variáveis qualitativas
Objetivo é avaliar se existe uma relação entre as variáveis qualitativas/categóricas e o grau de assosicação entre elas.

### Tabelas de distribuição conjunta de frequência

In [3]:
dados = pd.read_stata("/content/PlanoSaude.dta")
dados.head()

,operadora,satisfacao
0,total health,baixo
1,total health,baixo
2,total health,baixo
3,total health,baixo
4,total health,baixo


In [82]:
dados_freq = pd.DataFrame(dados.value_counts()).reset_index()
dados_freq.head(3)

,operadora,satisfacao,0
0,total health,baixo,40
1,mena saúde,médio,32
2,viva vida,baixo,32


In [125]:
#Filtrar por categoria
dados_freq_baixo  = dados_freq[dados_freq['satisfacao'] == 'baixo'].drop("satisfacao", axis = 1)
dados_freq_medio  = dados_freq[dados_freq['satisfacao'] == 'médio'].drop("satisfacao", axis = 1)
dados_freq_alto  = dados_freq[dados_freq['satisfacao'] == 'alto'].drop("satisfacao", axis = 1)

#Renomear coluna
dados_freq_baixo = dados_freq_baixo.rename(columns = {0:"Baixo"}, inplace = False)
dados_freq_medio = dados_freq_medio.rename(columns = {0:"Médio"}, inplace = False)
dados_freq_alto = dados_freq_alto.rename(columns = {0:"Alto"}, inplace = False)

#Merge dos dados
dados_freq_final = dados_freq_baixo
dados_freq_final = dados_freq_final.merge(dados_freq_medio, left_on='operadora', right_on='operadora')
dados_freq_final = dados_freq_final.merge(dados_freq_alto, left_on='operadora', right_on='operadora')

#Calcular o total por operador
dados_freq_final["Total"] = dados_freq_final["Baixo"] + dados_freq_final["Médio"] + dados_freq_final["Alto"]

dados_freq_final = dados_freq_final.append({'operadora':'Total',
                         'Baixo':sum(dados_freq_final["Baixo"]),
                         'Médio':sum(dados_freq_final["Médio"]),
                         'Alto':sum(dados_freq_final["Alto"]),
                         'Total':sum(dados_freq_final["Total"])}, ignore_index=True)

dados_freq_final

,operadora,Baixo,Médio,Alto,Total
0,total health,40,16,12,68
1,viva vida,32,24,16,72
2,mena saúde,24,32,4,60
3,Total,96,72,32,200


In [121]:
dados_freq.groupby(["operadora", "satisfacao"]).sum()

0
operadora    satisfacao    
total health baixo       40
             médio       16
             alto        12
viva vida    baixo       32
             médio       24
             alto        16
mena saúde   baixo       24
             médio       32
             alto         4

### Medidas de associação
As principais medidas representativas são:

* `Qui-Quadrado` $X^2$ utilizada para dados qualitativos.
* `Coeficiente Phi` é o coeficiente V de contigência dados nominais
* `Spearman` para dados ordinais

#### Qui-Quadrado
Mede a discrepãncia entre uma tabela de contigência observada e um tabela de contigência esperada.

$X^2 = \sum_{i=1}^I\sum_{j=1}^J\frac{(O_{ij} - E_{ij})^2}{E_{ij}}$